In [6]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
from dash import no_update
import datetime as dt
import folium
from dash import no_update
#Create app
app = dash.Dash(__name__)
#Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True
# Read the wildfire data into pandas dataframe
df =  pd.read_csv(r'C:\Users\mgich\OneDrive\Desktop\historical_automobile_sales.csv')
#Extract year and month from the date column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name() #used for the names of the months
df['Year'] = pd.to_datetime(df['Date']).dt.year
#Layout Section of Dash
#Task 1 Add the Title to the Dashboard
app.layout = html.Div(children=[html.H1('Automobile Sales Statistics Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),
# TASK 2: Add the radio items and a dropdown right below the first inner division
     #outer division starts
     html.Div([
                   # First inner divsion for  adding dropdown helper text for Selected Drive wheels
                    html.Div([
                            html.H2('yearly statistics:', style={'margin-right': '2em'}),

                    #Radio items to select the region
                    #dcc.RadioItems(['NSW','QL','SA','TA','VI','WA'], 'NSW', id='region',inline=True)]),
                    dcc.RadioItems([{"label":"Yearly Statistics","value": "Yearly Statistics"},
                                    {"label":"Recession Period Statistics","value": "Recession Period Statistics"}], id = 'select year',inline=True)]),
                    #Dropdown to select year
                    html.Div([
                            html.H2('Select Year:', style={'margin-right': '2em'}),
                        dcc.Dropdown(df.Year.unique(), value = 1980,id='Select year')
                    ]),
#TASK 3: Add two empty divisions for output inside the next inner division. 
         #Second Inner division for adding 2 inner divisions for 2 output graphs
                    html.Div([
                
                        html.Div([ ], id='select-year'),
                        html.Div([ ], id='output_container',className='chart-item',)
                    ], style={'display': 'flex'}),

    ])
    #outer division ends

])
#layout ends
#TASK 4: Add the Ouput and input components inside the app.callback decorator.
#Place to add @app.callback Decorator
@app.callback([Output(component_id='output-container', component_property='children')],
               [Input(component_id='dropdown-statistics', component_property='value'),
                Input(component_id='select year', component_property='value')])
#TASK 5: Add the callback function.   
#Place to define the callback function .
def update_output_container(selected_statistics, input_year):
      if selected_statistics == 'Recession Period Statistics':
          # Filter the data for recession periods
          recession_data = data[data['Recession'] == 1]

          #Plot 1 Automobile sales fluctuate over Recession Period (year wise)
          yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
          R_chart1 = dcc.Graph(
              figure=px.line(yearly_rec,
                  x='year',
                  y='automobile_sales',
                  title="Average Automobile Sales fluctuation over Recession Period"))

          #Plot 2 Calculate the average number of vehicles sold by vehicle type
          average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
          R_chart2  = dcc.Graph(
              figure=px.bar(average_sales,
                  x='vehicle type',y='automobile sales',
                  title= "Average Vehicle Sales for Each Vehicle Category"))

          # Plot pie chart for total expenditure share by vehicle type during recessions
          exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
          R_chart3 = dcc.Graph(
              figure=px.pie(exp_rec,
                values='advertising expenditure',
                  names='vehicle type',
                  title="Total Advertising Expenditure Share of Vehicle Type During Recession"
                  )
          )

          # Plot bar chart for the effect of unemployment rate on vehicle type and sales
          unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
          R_chart4 = dcc.Graph(
              figure=px.bar(
                  unemp_data,
                  x='unemployment rate',
                  y='Automobile sales',
                  color='vehicle type',
                  labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                  title="Effect of Unemployment Rate on Sales of various Vehicle Types")
          )
          return [
              html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
              html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})
              ]
      elif (input_year and selected_statistics=='Yearly Statistics') :
          yearly_data = data[data['Year'] == input_year]


          return [
                  html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),
                  html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                  ]

      else:
          return None

  # Run the Dash app
if __name__ == '__main__':
      app.run_server(debug=True)
